# Modules

In [2]:
import torch
import torch.nn as nn

In [6]:
# Creating a class OurModule with wrapped functions
class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 5),
            nn.ReLU(),
            nn.Linear(5, 20),
            nn.ReLU(),
            nn.Linear(20, num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.pipe(x)

In [7]:
net = OurModule(num_inputs = 2, num_classes = 3)
print(net)
v = torch.FloatTensor([[2, 3]])
out = net(v)
print(out)
print("Cuda's availability is %s" % torch.cuda.is_available())
if torch.cuda.is_available():
    print("Data from cuda: %s" % out.to('cuda'))

OurModule(
  (pipe): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Softmax(dim=1)
  )
)
tensor([[0.3319, 0.3350, 0.3332]], grad_fn=<SoftmaxBackward>)
Cuda's availability is True
Data from cuda: tensor([[0.3319, 0.3350, 0.3332]], device='cuda:0', grad_fn=<CopyBackwards>)


## Tensorboard
Tensorboard is used for viewing the performace of the model.

In [1]:
import math
from tensorboardX import SummaryWriter

In [3]:
writer = SummaryWriter()

funcs = {"sin": math.sin, "cos": math.cos, "tan": math.tan}

for angle in range(-360, 360):
    # Converting angle from degree to radians
    angle_rad = angle * math.pi/180
    for name, fun in funcs.items():
        val = fun(angle_rad)
        # storing name of Fucntion, its value and the angle
        writer.add_scalar(name, val, angle)

writer.close()

A folder named 'runs' will be created in the folder. In command prompt cd to the folder where runs is present and write the tensorboard command:
tensorboard --logdir PATH
in my case the command came out to be:
tensorboard --logdir runs\Aug13_17-55-29_Mohit

## Atari GAN

### Importing Packages

In [1]:
# For generating random values
import random
# for preprocessing images
import cv2

# for using pytorch
import torch
# for modules and sequence of neural network
import torch.nn as nn
# for using optim
import torch.optim as optim
# keeping the logs
from tensorboardX import SummaryWriter

# transformation of images
import torchvision.utils as vutils

# Initializing game environment and playing the game
import gym
import gym.spaces

# for creating arrays and perfoming calculations
import numpy as np

In [2]:
# Setting logger and initializing Variables
log = gym.logger
log.set_level(gym.logger.INFO)

LATENT_VECTOR_SIZE = 100
DISCR_FILTERS = 64
GENER_FILTERS = 64
BATCH_SIZE = 16

IMAGE_SIZE = 64

LEARNING_RATE = 0.0001
REPORT_EVERY_ITER = 100
SAVE_IMAGE_EVERY_ITER = 1000

In [3]:
class InputWrapper(gym.ObservationWrapper):
    '''
    Preprocessing of input numpy array:
    1. resizing image into predefined siize
    2. move color channel axis to a first place
    '''
    def __init__(self, *args):
        # Overriding the ObservationWrapper with InputWrapper
        super(InputWrapper, self).__init__(*args)
        # checking if type of observation space is same as that of gym space Box
        assert isinstance(self.observation_space, gym.spaces.Box)
        # saving previous observation space
        old_space = self.observation_space
        # Generating new observation space with same specs as old 
        self.observation_space = gym.spaces.Box(
            self.observation(old_space.low),
            self.observation(old_space.high),
            dtype=np.float32)
        
    def observation(self, observation):
        # resizing image
        new_obs = cv2.resize(
            observation, (IMAGE_SIZE, IMAGE_SIZE))
        # transforming (210, 160, 3) -> (3, 210, 160)
        new_obs = np.moveaxis(new_obs, 2, 0)
        return new_obs.astype(np.float32)

In [4]:
class Discriminator(nn.Module):
    '''
    Discriminator checks whether the image generated by generator is according to the given requirement or not
    '''
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()
        # creating a model to converge images into a single number
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=input_shape[0], out_channels=DISCR_FILTERS,
                      kernel_size=4, stride=2, padding=1), #(64, 32, 32)
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS, out_channels=DISCR_FILTERS*2,
                      kernel_size=4, stride=2, padding=1), #(128, 16, 16)
            nn.BatchNorm2d(DISCR_FILTERS*2),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS*2, out_channels=DISCR_FILTERS*4,
                      kernel_size=4, stride=2, padding=1), #(256, 8, 8)
            nn.BatchNorm2d(DISCR_FILTERS*4),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS*4, out_channels=DISCR_FILTERS*8,
                      kernel_size=4, stride=2, padding=1), #(512, 4, 4)
            nn.BatchNorm2d(DISCR_FILTERS*8),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS*8, out_channels=1,
                      kernel_size=4, stride=1, padding=0), #(1, 1, 1)
            nn.Sigmoid()
        )
        
    
    def forward(self, x):
        conv_out = self.model(x)
        return conv_out.view(-1, 1).squeeze(dim=1)

In [5]:
class Generator(nn.Module):
    def __init__(self, output_shape):
        super(Generator, self).__init__()
        # model deconvolves input vector into (3, 64, 64) image
        self.model = nn.Sequential(
            nn.ConvTranspose2d(in_channels=LATENT_VECTOR_SIZE, out_channels=GENER_FILTERS*8,
                               kernel_size=4, stride=1, padding=0), #(512, 4, 4)
            nn.BatchNorm2d(GENER_FILTERS*8),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS*8, out_channels=GENER_FILTERS*4,
                               kernel_size=4, stride=2, padding=1), #(256, 8, 8)
            nn.BatchNorm2d(GENER_FILTERS*4),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS*4, out_channels=GENER_FILTERS*2,
                               kernel_size=4, stride=2, padding=1), #(128, 16, 16)
            nn.BatchNorm2d(GENER_FILTERS*2),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS*2, out_channels=GENER_FILTERS,
                               kernel_size=4, stride=2, padding=1), #(64, 32, 32)
            nn.BatchNorm2d(GENER_FILTERS),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS, out_channels=output_shape[0],
                               kernel_size=4, stride=2, padding=1), #(3, 64, 64)
            nn.Tanh()
        )
        
    def forward(self, x):
        return self.model(x)

In [6]:
def iterate_batches(envs, batch_size=BATCH_SIZE):
    '''
    playing and geting observation for batch_size
    '''
    batch=[e.reset() for e in envs]
    env_gen = iter(lambda: random.choice(envs), None)
    
    while True:
        e = next(env_gen)
        obs, reward, is_done, _ = e.step(e.action_space.sample())
        if np.mean(obs) > 0.01:
            batch.append(obs)
        if len(batch) == batch_size:
            # Normalizing input between -1 to 1
            batch_np = np.array(batch, dtype=np.float32) * 2.0 / 255.0 - 1.0
            yield torch.tensor(batch_np)
            batch.clear()
        if is_done:
            e.reset()

In [7]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
envs = [
    InputWrapper(gym.make(name))
    for name in ('Breakout-v0', 'AirRaid-v0', 'Pong-v0')
]
input_shape = envs[0].observation_space.shape
print(input_shape)

INFO: Making new env: Breakout-v0
INFO: Making new env: AirRaid-v0
INFO: Making new env: Pong-v0
(3, 64, 64)


In [8]:
net_discr = Discriminator(input_shape=input_shape).to(device)
net_gener = Generator(output_shape=input_shape).to(device)

# Loss fucntion of Binary Cross Entropy
objective = nn.BCELoss()
# Optimizer for both generator and Discriminator
gen_optimizer = optim.Adam(
    params = net_gener.parameters(), lr=LEARNING_RATE,
    betas=(0.5, 0.999))
dis_optimizer = optim.Adam(
    params = net_discr.parameters(), lr=LEARNING_RATE,
    betas=(0.5, 0.999))

# Tensorboard logs
writer = SummaryWriter()

In [9]:
gen_losses = []
dis_losses = []
iter_no = 0

true_labels_v = torch.ones(BATCH_SIZE, device=device)
fake_labels_v = torch.zeros(BATCH_SIZE, device=device)

In [10]:
for batch_v in iterate_batches(envs):
    # fake samples, input is 4D: batch, filters, x, y
    gen_input_v = torch.FloatTensor(
        BATCH_SIZE, LATENT_VECTOR_SIZE, 1, 1)
    gen_input_v.normal_(0, 1)
    gen_input_v = gen_input_v.to(device)
    batch_v = batch_v.to(device)
    gen_output_v = net_gener(gen_input_v)
    
    # train discriminator
    dis_optimizer.zero_grad()
    dis_output_true_v = net_discr(batch_v)
    dis_output_fake_v = net_discr(gen_output_v.detach())
    dis_loss = objective(dis_output_true_v, true_labels_v) + \
               objective(dis_output_fake_v, fake_labels_v)
    dis_loss.backward()
    dis_optimizer.step()
    dis_losses.append(dis_loss.item())
    
    #train generator
    gen_optimizer.zero_grad()
    dis_output_v = net_discr(gen_output_v)
    gen_loss_v = objective(dis_output_v, true_labels_v)
    gen_loss_v.backward()
    gen_optimizer.step()
    gen_losses.append(gen_loss_v.item())
    
    iter_no += 1
    if iter_no % REPORT_EVERY_ITER == 0:
        log.info("Iter %d: gen_loss=%.3e, dis_loss=%.3e",
                 iter_no, np.mean(gen_losses),
                 np.mean(dis_losses))
        writer.add_scalar(
            "gen_loss", np.mean(gen_losses), iter_no)
        writer.add_scalar(
            "dis_loss", np.mean(dis_losses), iter_no)
        gen_losses = []
        dis_losses = []
    if iter_no % SAVE_IMAGE_EVERY_ITER == 0:
        writer.add_image("fake", vutils.make_grid(
            gen_output_v.data[:64], normalize=True), iter_no)
        writer.add_image("true", vutils.make_grid(
            batch_v.data[:64], normalize=True), iter_no)

INFO: Iter 100: gen_loss=5.561e+00, dis_loss=4.671e-02
INFO: Iter 200: gen_loss=7.136e+00, dis_loss=4.518e-03
INFO: Iter 300: gen_loss=7.748e+00, dis_loss=2.406e-03
INFO: Iter 400: gen_loss=8.021e+00, dis_loss=2.322e-03
INFO: Iter 500: gen_loss=8.110e+00, dis_loss=1.083e-03
INFO: Iter 600: gen_loss=8.188e+00, dis_loss=1.546e-01
INFO: Iter 700: gen_loss=7.006e+00, dis_loss=1.238e-02
INFO: Iter 800: gen_loss=6.947e+00, dis_loss=6.379e-03
INFO: Iter 900: gen_loss=6.979e+00, dis_loss=3.768e-03
INFO: Iter 1000: gen_loss=8.033e+00, dis_loss=2.739e-03
INFO: Iter 1100: gen_loss=8.535e+00, dis_loss=2.205e-03
INFO: Iter 1200: gen_loss=7.516e+00, dis_loss=2.126e-03
INFO: Iter 1300: gen_loss=7.801e+00, dis_loss=1.084e-03
INFO: Iter 1400: gen_loss=8.568e+00, dis_loss=1.603e-02
INFO: Iter 1500: gen_loss=1.154e+01, dis_loss=3.796e-02
INFO: Iter 1600: gen_loss=7.052e+00, dis_loss=8.256e-02
INFO: Iter 1700: gen_loss=6.813e+00, dis_loss=9.074e-03
INFO: Iter 1800: gen_loss=6.462e+00, dis_loss=1.195e-01
I

KeyboardInterrupt: 

The GAN is trained for 55,000 steps and generates pretty good results. It can be checked using tensorboard.

## Atari Gan Ignite

In [1]:
import random
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
#import ignite
from ignite.engine import Engine, Events
from ignite.metrics import RunningAverage
from ignite.contrib.handlers import tensorboard_logger as tb_logger

import torchvision.utils as vutils

import gym
import gym.spaces

import numpy as np

In [2]:
# tHIS PART IS SAME AS ABOVE
log = gym.logger
log.set_level(gym.logger.INFO)

LATENT_VECTOR_SIZE = 100
DISCR_FILTERS = 64
GENER_FILTERS = 64
BATCH_SIZE = 16

IMAGE_SIZE = 64

LEARNING_RATE = 0.0001
REPORT_EVERY_ITER = 100
SAVE_IMAGE_EVERY_ITER = 1000

In [3]:
# SAME AS BEFORE
class InputWrapper(gym.ObservationWrapper):
    '''
    Preprocessing of input numpy array:
    1. resizing image into predefined siize
    2. move color channel axis to a first place
    '''
    def __init__(self, *args):
        # Overriding the ObservationWrapper with InputWrapper
        super(InputWrapper, self).__init__(*args)
        # checking if type of observation space is same as that of gym space Box
        assert isinstance(self.observation_space, gym.spaces.Box)
        # saving previous observation space
        old_space = self.observation_space
        # Generating new observation space with same specs as old 
        self.observation_space = gym.spaces.Box(
            self.observation(old_space.low),
            self.observation(old_space.high),
            dtype=np.float32)
        
    def observation(self, observation):
        # resizing image
        new_obs = cv2.resize(
            observation, (IMAGE_SIZE, IMAGE_SIZE))
        # transforming (210, 160, 3) -> (3, 210, 160)
        new_obs = np.moveaxis(new_obs, 2, 0)
        return new_obs.astype(np.float32)

In [4]:
# SAME AS BEFORE
class Discriminator(nn.Module):
    '''
    Discriminator checks whether the image generated by generator is according to the given requirement or not
    '''
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()
        # creating a model to converge images into a single number
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=input_shape[0], out_channels=DISCR_FILTERS,
                      kernel_size=4, stride=2, padding=1), #(64, 32, 32)
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS, out_channels=DISCR_FILTERS*2,
                      kernel_size=4, stride=2, padding=1), #(128, 16, 16)
            nn.BatchNorm2d(DISCR_FILTERS*2),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS*2, out_channels=DISCR_FILTERS*4,
                      kernel_size=4, stride=2, padding=1), #(256, 8, 8)
            nn.BatchNorm2d(DISCR_FILTERS*4),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS*4, out_channels=DISCR_FILTERS*8,
                      kernel_size=4, stride=2, padding=1), #(512, 4, 4)
            nn.BatchNorm2d(DISCR_FILTERS*8),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS*8, out_channels=1,
                      kernel_size=4, stride=1, padding=0), #(1, 1, 1)
            nn.Sigmoid()
        )
        
    
    def forward(self, x):
        conv_out = self.model(x)
        return conv_out.view(-1, 1).squeeze(dim=1)

In [5]:
# SAME AS BEFORE
class Generator(nn.Module):
    def __init__(self, output_shape):
        super(Generator, self).__init__()
        # model deconvolves input vector into (3, 64, 64) image
        self.model = nn.Sequential(
            nn.ConvTranspose2d(in_channels=LATENT_VECTOR_SIZE, out_channels=GENER_FILTERS*8,
                               kernel_size=4, stride=1, padding=0), #(512, 4, 4)
            nn.BatchNorm2d(GENER_FILTERS*8),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS*8, out_channels=GENER_FILTERS*4,
                               kernel_size=4, stride=2, padding=1), #(256, 8, 8)
            nn.BatchNorm2d(GENER_FILTERS*4),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS*4, out_channels=GENER_FILTERS*2,
                               kernel_size=4, stride=2, padding=1), #(128, 16, 16)
            nn.BatchNorm2d(GENER_FILTERS*2),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS*2, out_channels=GENER_FILTERS,
                               kernel_size=4, stride=2, padding=1), #(64, 32, 32)
            nn.BatchNorm2d(GENER_FILTERS),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS, out_channels=output_shape[0],
                               kernel_size=4, stride=2, padding=1), #(3, 64, 64)
            nn.Tanh()
        )
        
    def forward(self, x):
        return self.model(x)

In [6]:
# SAME AS BEFORE
def iterate_batches(envs, batch_size=BATCH_SIZE):
    '''
    playing and geting observation for batch_size
    '''
    batch=[e.reset() for e in envs]
    env_gen = iter(lambda: random.choice(envs), None)
    
    while True:
        e = next(env_gen)
        obs, reward, is_done, _ = e.step(e.action_space.sample())
        if np.mean(obs) > 0.01:
            batch.append(obs)
        if len(batch) == batch_size:
            # Normalizing input between -1 to 1
            batch_np = np.array(batch, dtype=np.float32) * 2.0 / 255.0 - 1.0
            yield torch.tensor(batch_np)
            batch.clear()
        if is_done:
            e.reset()

In [7]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
envs = [
    InputWrapper(gym.make(name))
    for name in ('Breakout-v0', 'AirRaid-v0', 'Pong-v0')
]
input_shape = envs[0].observation_space.shape

net_discr = Discriminator(input_shape=input_shape).to(device)
net_gener = Generator(output_shape=input_shape).to(device)

# Loss fucntion of Binary Cross Entropy
objective = nn.BCELoss()
# Optimizer for both generator and Discriminator
gen_optimizer = optim.Adam(
    params = net_gener.parameters(), lr=LEARNING_RATE,
    betas=(0.5, 0.999))
dis_optimizer = optim.Adam(
    params = net_discr.parameters(), lr=LEARNING_RATE,
    betas=(0.5, 0.999))

true_labels_v = torch.ones(BATCH_SIZE, device=device)
fake_labels_v = torch.zeros(BATCH_SIZE, device=device)

INFO: Making new env: Breakout-v0
INFO: Making new env: AirRaid-v0
INFO: Making new env: Pong-v0


In [18]:
def process_batch(trainer, batch):
    # fake samples, input is 4D: batch, filters, x, y
    gen_input_v = torch.FloatTensor(
        BATCH_SIZE, LATENT_VECTOR_SIZE, 1, 1)
    gen_input_v.normal_(0, 1)
    gen_input_v = gen_input_v.to(device)
    batch_v = batch.to(device)
    gen_output_v = net_gener(gen_input_v)
    
    # train discriminator
    dis_optimizer.zero_grad()
    dis_output_true_v = net_discr(batch_v)
    dis_output_fake_v = net_discr(gen_output_v.detach())
    dis_loss = objective(dis_output_true_v, true_labels_v) + \
               objective(dis_output_fake_v, fake_labels_v)
    dis_loss.backward()
    dis_optimizer.step()
    
    #train generator
    gen_optimizer.zero_grad()
    dis_output_v = net_discr(gen_output_v)
    gen_loss = objective(dis_output_v, true_labels_v)
    gen_loss.backward()
    gen_optimizer.step()
    
    if trainer.state.iteration % SAVE_IMAGE_EVERY_ITER == 0:
        fake_img = vutils.make_grid(
            gen_output_v.data[:64], normalize=True)
        trainer.tb.writer.add_image(
            "fake", fake_img, trainer.state.iteration)
        real_img = vutils.make_grid(
            batch_v.data[:64], normalize=True)
        trainer.tb.writer.add_image(
            "real", real_img, trainer.state.iteration)
        trainer.tb.writer.flush()
    return dis_loss.item(), gen_loss.item()

In [19]:
engine = Engine(process_batch)
tb = tb_logger.TensorboardLogger(log_dir=None)
engine.tb = tb
RunningAverage(output_transform=lambda out: out[1]).\
    attach(engine, "avg_loss_gen")
RunningAverage(output_transform=lambda out: out[0]).\
    attach(engine, "avg_loss_dis")

handler = tb_logger.OutputHandler(tag="train",
                                  metric_names=['avg_loss_gen', 'avg_loss_dis'])
tb.attach(engine, log_handler=handler,
          event_name=Events.ITERATION_COMPLETED)

In [20]:
@engine.on(Events.ITERATION_COMPLETED)
def log_losses(trainer):
    if trainer.state.iteration % REPORT_EVERY_ITER == 0:
        log.info("%d: gen_loss=%f, dis_loss=%f",
                 trainer.state.iteration,
                 trainer.state.metrics['avg_loss_gen'],
                 trainer.state.metrics['avg_loss_dis'])

engine.run(data=iterate_batches(envs))

INFO: 100: gen_loss=4.975398, dis_loss=0.387582
INFO: 200: gen_loss=4.352178, dis_loss=0.353541
INFO: 300: gen_loss=4.789439, dis_loss=0.107345
INFO: 400: gen_loss=4.876387, dis_loss=0.127333
INFO: 500: gen_loss=5.258148, dis_loss=0.228796
INFO: 600: gen_loss=5.414878, dis_loss=0.272098
INFO: 700: gen_loss=5.240187, dis_loss=0.235681
INFO: 800: gen_loss=4.576335, dis_loss=0.255500
INFO: 900: gen_loss=5.885434, dis_loss=0.147684
INFO: 1000: gen_loss=5.916265, dis_loss=0.089216
INFO: 1100: gen_loss=5.243893, dis_loss=0.175737
INFO: 1200: gen_loss=6.136098, dis_loss=0.044893
INFO: 1300: gen_loss=6.990434, dis_loss=0.022466
INFO: 1400: gen_loss=5.745535, dis_loss=0.074429
INFO: 1500: gen_loss=5.623897, dis_loss=0.043017
INFO: 1600: gen_loss=6.508416, dis_loss=0.036814
INFO: 1700: gen_loss=5.394989, dis_loss=0.224614
INFO: 1800: gen_loss=6.523425, dis_loss=0.049758
INFO: 1900: gen_loss=7.108238, dis_loss=0.012480
INFO: 2000: gen_loss=7.346043, dis_loss=0.028708
INFO: 2100: gen_loss=7.930427

KeyboardInterrupt: 

Trained the Neural Network for 58K steps ans the result 